In [98]:
import numpy as np
import torch as tf
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.metrics import classification_report

from FERData import FERDataset

# Device configuration
device = tf.device('cuda' if tf.cuda.is_available() else 'cpu')

In [8]:
train_dataset = FERDataset('./dataset/train')
test_dataset =  FERDataset('./dataset/test')

In [9]:
train_dataLoader = DataLoader(train_dataset, 64, shuffle=True)
test_dataLoader = DataLoader(test_dataset, 64, shuffle=True)



In [80]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(6*6*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        x = x[:,None,:,:]
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [81]:
def train(dataLoader, model, optimizer, lossf):
  model.train()
  losses = np.array([])

  for img, label in dataLoader:

    optimizer.zero_grad()
    t_imgs = img.to(device, dtype=tf.float)
    t_labels = label.to(device, dtype=tf.int64)

    prediction = model(t_imgs)
    loss = lossf(prediction, t_labels)
    losses = np.append(losses, loss.item())

    loss.backward()

    optimizer.step()

  return np.average(losses)



In [96]:
def test(dataLoader, model, optimizer, lossf):
  model.eval()
  losses, predictions, labels = np.array([]), np.array([]), np.array([])

  for img, label in dataLoader:

    t_imgs = img.to(device, dtype=tf.float)
    t_labels = label.to(device, dtype=tf.int64)
    labels = np.append(labels, t_labels.cpu())

    output = model(t_imgs)
    _, prediction = tf.max(output, 1)
    predictions = np.append(predictions, prediction.cpu())

    loss = lossf(output, t_labels)
    losses = np.append(losses, loss.item())

  return np.average(losses), predictions, labels

In [90]:
model = VGG16(7).to(device)
optimizer = Adam(model.parameters(),lr = 0.0001)
lossf = nn.CrossEntropyLoss()
epochs = 50

def run(train_dataLoader, test_dataLoader, model, optimizer, lossf, epochs):
  for epoch in range(epochs):
    avgLosses = train(train_dataLoader, model, optimizer, lossf)
    print(f'{epoch} - {avgLosses}')

  test_avgLosess, test_predictions, test_labels = test(test_dataLoader, model, optimizer, lossf)
  print(classification_report(test_labels, test_predictions, digits=4))

In [99]:
test_avgLosess, test_predictions, test_labels = test(test_dataLoader, model, optimizer, lossf)
print(classification_report(test_labels, test_predictions, digits=4))

              precision    recall  f1-score   support

         0.0     0.8475    0.4505    0.5882       111
         1.0     0.4369    0.5068    0.4693      1024
         2.0     0.5579    0.5239    0.5404      1233
         3.0     0.4752    0.5992    0.5300       958
         4.0     0.7755    0.7401    0.7574       831
         5.0     0.8468    0.7570    0.7994      1774
         6.0     0.4882    0.4643    0.4760      1247

    accuracy                         0.6027      7178
   macro avg     0.6326    0.5774    0.5944      7178
weighted avg     0.6185    0.6027    0.6075      7178



In [91]:
run(train_dataLoader, test_dataLoader, model, optimizer, lossf, epochs)


0 - 1.5040892617474155
1 - 1.2129047438668248
2 - 1.0764052184228112
3 - 0.9622338082317785
4 - 0.8401948691476426
5 - 0.7178220534510495
6 - 0.5807699125699848
7 - 0.45653169232115715
8 - 0.3566974802876103
9 - 0.27729703329984756
10 - 0.22232931795922048
11 - 0.18883200516646317
12 - 0.16176331801991287
13 - 0.14107558034559534
14 - 0.13098028697745506
15 - 0.1195956622219265
16 - 0.1122997846694991
17 - 0.1004532041144298
18 - 0.10094071255026091
19 - 0.10125539098909843
20 - 0.08189186854661341
21 - 0.09597589716290379
22 - 0.0865359755008915
23 - 0.08218656140970759
24 - 0.07323813370147923
25 - 0.07138170754456091
26 - 0.0853114510432118
27 - 0.07438209632858593
28 - 0.0650492251922225
29 - 0.06646673043698222
30 - 0.07329824571767386
31 - 0.06655669813448842
32 - 0.06219976771734904
33 - 0.06385919933138884
34 - 0.060158184256595464
35 - 0.057373369283474286
36 - 0.05685246173063716
37 - 0.061593306488367854
38 - 0.05296063635919351
39 - 0.05102250905409829
40 - 0.05042168835993

RuntimeError: ignored

In [1]:
tf.save(model.state_dict(), '/content/model01')

NameError: name 'tf' is not defined